In [56]:
import os
import sys
import torch
import pymongo
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt

from deep_fields import data_path
from deep_fields.models.crypto.birth_stats import top_and_birth
from deep_fields.data.crypto.data_basics import top_from_collection
from deep_fields.data.crypto.create_dataset_files import create_merged_dataframe

%matplotlib inline 

In [101]:
date0 = datetime(2018, 1, 1)
datef = datetime(2019, 1, 1)

date = "2021-06-14"
span ="full"
top=10
date0=date0
datef=datef
max_size=500

final_date = datetime.strptime(date,"%Y-%m-%d")
crypto_folder = os.path.join(data_path, "raw", "crypto")
data_folder = os.path.join(crypto_folder, date)

In [102]:
client = pymongo.MongoClient()
db = client["crypto"]
collection = db['birth_{0}'.format(date)]

top_coins_name, coins_name = top_and_birth(date, date0, datef, top=10, max_size=max_size)


data_merged, coins_data = create_merged_dataframe(data_folder,
                                                  collection,
                                                  break_point=max_size,
                                                  all_coins_ids=coins_name,
                                                  span=span)

survival_time = []
birth_dates = []
for a in coins_data:
    survival_time.append(a["survival_time"])
    birth_dates.append(a["birth_date"])

Current Coin 1 ethereum
Current Coin 2 tether
Current Coin 3 binancecoin
Current Coin 4 cardano
Current Coin 5 dogecoin
Current Coin 6 ripple
Current Coin 7 bitcoin-cash
Current Coin 8 litecoin
Current Coin 9 chainlink
Current Coin 10 matryx
Current Coin 11 storm
Current Coin 12 cryptopay
Current Coin 13 appcoins
Current Coin 14 time-new-bank
Current Coin 15 pylon-network
Current Coin 16 inflationcoin
Current Coin 17 aelf
Current Coin 18 wax
Current Coin 19 nekonium
Current Coin 20 exrnchain
Current Coin 21 canyacoin
Current Coin 22 womencoin
Current Coin 23 united-traders-token
Current Coin 24 bread
Current Coin 25 zap
Current Coin 26 medibloc
Current Coin 27 spankchain
Current Coin 28 selfkey
Current Coin 29 lightning-bitcoin
Current Coin 30 karbo
Current Coin 31 bitclave
Current Coin 32 valorbit
Current Coin 33 nebulas
Current Coin 34 mixin
Current Coin 35 htmlcoin
Current Coin 36 echolink
Current Coin 37 casinocoin
Current Coin 38 singularitynet
Current Coin 39 polis
Current Coin 4

Current Coin 312 blockmesh-2
Current Coin 313 fsn
Current Coin 314 parkgene
Current Coin 315 newton-coin-project
Current Coin 316 crowd-machine
Current Coin 317 pixie-coin
Current Coin 318 skrumble-network
Current Coin 319 fuzex
Current Coin 320 daostack
Current Coin 321 bezant
Current Coin 322 tokenstars-team
Current Coin 323 digix-gold
Current Coin 324 penta
Current Coin 325 mytoken
Current Coin 326 loki-network
Current Coin 327 daex
Current Coin 328 bitcoin-interest
Current Coin 329 medical-token-currency
Current Coin 330 key
Current Coin 331 sakura-bloom
Current Coin 332 cnn
Current Coin 333 balkan-coin
Current Coin 334 unibright
Current Coin 335 ucot
Current Coin 336 taler
Current Coin 337 relex
Current Coin 338 npcoin
Current Coin 339 cryply
Current Coin 340 bitcoin-w-spectrum
Current Coin 341 zippie
Current Coin 342 open-platform
Current Coin 343 morpheus-network
Current Coin 344 libra-credit
Current Coin 345 intelligent-investment-chain
Current Coin 346 cybereits
Current Coin 3

In [103]:
data_merged = data_merged.loc[:final_date]
initial_date = data_merged.index[0]

In [104]:
#define prices and returns
price = torch.Tensor(data_merged["price"].values).T
market_cap = torch.Tensor(data_merged["market_cap"].values).T
volume = torch.Tensor(data_merged["volume"].values).T
portfolio_pmv = torch.cat([price.unsqueeze(-1), market_cap.unsqueeze(-1), volume.unsqueeze(-1)], dim=-1)

In [105]:
birth_index = torch.Tensor([(birth_date-initial_date).days for birth_date in birth_dates]).long()

In [106]:
initial_prices = torch.index_select(price,1,birth_index)
initial_prices = torch.diagonal(initial_prices)

In [117]:
not_nans = price == price

In [126]:
death_index = torch.Tensor([torch.where(not_nan)[0].max() for not_nan in not_nans]).long()

In [127]:
final_prices = torch.index_select(price,1,death_index)
final_prices = torch.diagonal(final_prices)

In [128]:
final_prices

tensor([3.9148e+04, 2.5178e+03, 9.9943e-01, 3.6525e+02, 1.5668e+00, 3.2504e-01,
        8.8603e-01, 6.1275e+02, 1.7135e+02, 2.3398e+01, 4.7956e-02, 2.1589e-02,
        8.3481e-02, 7.4069e-02, 2.8875e-03, 7.1867e-01, 7.9743e-06, 2.1789e-01,
        1.3554e-01, 1.5600e-03, 5.4226e-05, 1.2376e-01, 1.2869e-05, 5.5436e-01,
        1.5180e-01, 7.5402e-02, 4.8985e-02, 1.6517e-02, 1.1406e-02, 1.0333e+00,
        1.4150e-01, 6.9047e-04, 7.9735e-07, 4.3089e-01, 3.4212e+02, 1.5890e-04,
        2.4232e-03, 2.8221e-03, 2.1197e-01, 1.5083e-01, 9.9971e-03, 8.5370e-03,
        8.5027e-02, 1.7268e-03, 3.0218e-03, 2.0346e-02, 7.6979e-05, 2.6751e-02,
        1.0640e-01, 4.6741e-03, 1.6206e-01, 4.9912e-02, 2.6949e-02, 2.8075e-02,
        3.9141e-03, 1.4059e-01, 2.2096e-03, 5.1509e-02, 3.1166e-03, 7.9087e-03,
        5.3744e-03, 1.6018e-01, 1.2374e-03, 8.7603e+00, 2.8420e-03, 6.5592e-03,
        5.0699e-05, 3.6798e-03, 7.4404e-01, 1.5301e-02, 1.0978e-04, 4.7548e-02,
        1.0371e-03, 1.1378e-02, 7.1827e-

In [ ]:
for select_series in range(price.shape[0]):
    series = pd.DataFrame(data_merged["price"].values[:,select_series],index=data_merged.index)
    print(series.loc[birth_dates[select_series],0])

In [28]:
negative_survival_index = list(map(lambda a: a * (-1), survival_time))
birth_index = portfolio_pmv[:, :, 0].shape[1] + torch.Tensor(negative_survival_index).long()
initial_prices = torch.diagonal(torch.index_select(portfolio_pmv[:,:,0],1,birth_index))

In [29]:
initial_prices

tensor([2.8657e+02,        nan,        nan, 1.7349e-01, 3.0300e-02, 1.6782e-04,
        2.2270e-02, 2.3462e+02, 2.1642e+00, 1.8122e-01, 8.0344e-01, 1.4111e-01,
        3.4278e-01, 3.9060e+00, 2.7322e-01, 2.9620e+01, 1.0164e-04, 2.6041e+00,
        2.0119e+00])

In [ ]:
portfolio_survival = data_loader.portfolio_survival - skip_end
prices = data_loader.portfolio_pmv[:,:,0]
X = torch.log(prices)
log_initial_prices = torch.index_select(X,1,data_loader.birth_index+1)
log_initial_prices = torch.diagonal(log_initial_prices)
log_final_prices = X[:,-skip_end]